In [1]:
# Import the library
import bitfinex
import datetime
import time
import pandas as pd
from pathlib import Path

#### Step 1 - Define Query parameters: Ticker, Start / End Date

In [2]:
# Define query parameters

# Create list of currency pair of interest
pair = 'BTCUSD' 

# Define timeframe of data to pull
TIMEFRAME = '1h'#,'4h','1h','15m','1m'
TIMEFRAME_S = 3600 # seconds in TIMEFRAME

# Define the start date
t_start = datetime.datetime(2017, 1, 1, 0, 0)
t_start = time.mktime(t_start.timetuple()) * 1000

# Define the end date
t_stop = datetime.datetime.now()
t_stop = time.mktime(t_stop.timetuple()) * 1000

#### Step 2 - Create api instance from Bitfinex library, and fetch data for longer time intervals, current limit for Bitfinex API is 60 calls per minute

In [3]:
# Fetch data
def fetch_data(start, stop, symbol, interval, TIMEFRAME_S):
    limit = 1000    # We want the maximum of 1000 data points
    
    # Create api instance
    api_v2 = bitfinex.bitfinex_v2.api_v2()
    hour = TIMEFRAME_S * 1000
    step = hour * limit
    data = []

    total_steps = (stop-start)/hour
    while total_steps > 0:
        if total_steps < limit: # recalculating ending steps
            step = total_steps * hour

        end = start + step
        data += api_v2.candles(symbol=symbol, interval=interval, limit=limit, start=start, end=end)
        print(pd.to_datetime(start, unit='ms'), pd.to_datetime(end, unit='ms'), "steps left:", total_steps)
        start = start + step
        total_steps -= limit
        time.sleep(1.5)
    return data

result = fetch_data(t_start, t_stop, pair, TIMEFRAME, TIMEFRAME_S)

No keys, only access to public API functions
2017-01-01 06:00:00 2017-02-11 22:00:00 steps left: 42755.67805555555
2017-02-11 22:00:00 2017-03-25 14:00:00 steps left: 41755.67805555555
2017-03-25 14:00:00 2017-05-06 06:00:00 steps left: 40755.67805555555
2017-05-06 06:00:00 2017-06-16 22:00:00 steps left: 39755.67805555555
2017-06-16 22:00:00 2017-07-28 14:00:00 steps left: 38755.67805555555
2017-07-28 14:00:00 2017-09-08 06:00:00 steps left: 37755.67805555555
2017-09-08 06:00:00 2017-10-19 22:00:00 steps left: 36755.67805555555
2017-10-19 22:00:00 2017-11-30 14:00:00 steps left: 35755.67805555555
2017-11-30 14:00:00 2018-01-11 06:00:00 steps left: 34755.67805555555
2018-01-11 06:00:00 2018-02-21 22:00:00 steps left: 33755.67805555555
2018-02-21 22:00:00 2018-04-04 14:00:00 steps left: 32755.678055555552
2018-04-04 14:00:00 2018-05-16 06:00:00 steps left: 31755.678055555552
2018-05-16 06:00:00 2018-06-26 22:00:00 steps left: 30755.678055555552
2018-06-26 22:00:00 2018-08-07 14:00:00 st

#### Create dataframe and save to csv file

In [4]:
# Define columns name
names = ['Date', 'Open', 'Close', 'High', 'Low', 'Volume']

# Create dataframe
df = pd.DataFrame(result, columns=names)
df.drop_duplicates(inplace=True)
df['Date'] = pd.to_datetime(df['Date'], unit='ms')
df.set_index('Date', inplace=True)
df.sort_index(inplace=True)

# Save dataframe to csv file
df.to_csv(f"{pair}_{TIMEFRAME}.csv")